In [1]:
import pandas as pd
import os, glob
import numpy as np
from sklearn.cross_validation import train_test_split
import tensorflow as tf

C:\Users\Admin\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
data = pd.read_csv('C:/Users/Admin/Desktop/CWD/DATA/MNIST REAL DATA/train.csv')
data.head()

,filename,label
0,0.png,4
1,1.png,9
2,2.png,1
3,3.png,7
4,4.png,3


In [3]:
import os.path
from skimage import io
cwd = 'C:/Users/Admin/Desktop/CWD/DATA/MNIST REAL DATA/Images/train'
X = data['filename']
Y = data['label']
x = []
for image in X:
    x.append(os.path.join(cwd,image).replace("\\","/"))
x = np.array(x)


all_images = []
for image_path in x:
  img = io.imread(image_path , as_grey=True)
  img = img.reshape(img.shape[0]*img.shape[1],)
  all_images.append(img)
x_train = np.array(all_images)
y_train = np.array(Y)


In [4]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder


onehot_encoder = OneHotEncoder(sparse=False)
y_train = y_train.reshape(len(y_train), 1)
y_train = onehot_encoder.fit_transform(y_train)
print(y_train)
print(x_train.shape)
print(y_train.shape)

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  1.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  1.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]]
(49000, 784)
(49000, 10)


In [5]:
flat_image_size = x_train[1].shape[0]
no_of_classes = 10
n_h1 = 500
n_h2 = 450
n_h3 = 212
n_out = no_of_classes
x_train.shape
# y_train.shape

(49000, 784)

In [6]:
from sklearn.cross_validation import train_test_split
x_train1, x_test1, y_train1, y_test1 = train_test_split(x_train, y_train, test_size = 0.2)
type(x_test1)

numpy.ndarray

In [7]:
x = tf.placeholder('float', shape = [None, flat_image_size])
y = tf.placeholder('float', shape = [None, no_of_classes])

def neural_nets(data):
    hidden_layer_1 = {'weights': tf.Variable(tf.random_normal([flat_image_size, n_h1])),
                      'bias': tf.Variable(tf.random_normal([n_h1]))}
    
    hidden_layer_2 = {'weights': tf.Variable(tf.random_normal([n_h1, n_h2])),
                      'bias': tf.Variable(tf.random_normal([n_h2]))}
    
    hidden_layer_3 = {'weights': tf.Variable(tf.random_normal([n_h2, n_h3])),
                      'bias': tf.Variable(tf.random_normal([n_h3]))}
    
    output = {'weights': tf.Variable(tf.random_normal([n_h3, n_out])),
              'bias': tf.Variable(tf.random_normal([n_out]))}
    
    l1 = tf.add(tf.matmul(x,hidden_layer_1['weights']), hidden_layer_1['bias'])
    l1 = tf.nn.relu(l1)

    l2 = tf.add(tf.matmul(l1,hidden_layer_2['weights']), hidden_layer_2['bias'])
    l2 = tf.nn.relu(l2)
    
    l3 = tf.add(tf.matmul(l2,hidden_layer_3['weights']), hidden_layer_3['bias'])
    l3 = tf.nn.relu(l3)
    
    output = tf.add(tf.matmul(l3,output['weights']), output['bias'])
    output = tf.nn.relu(output)

    return output


In [8]:
def neural_nets_train(data):
    prediction = neural_nets(data)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = prediction, labels = y))
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    no_epoch = 282
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for epoch in range(no_epoch):
            epoch_loss = 0
            o,c = sess.run([optimizer, cost], feed_dict = {x:x_train1, y:y_train1})
            epoch_loss +=c
            print('Epoch',epoch,'out of ',no_epoch,'Epochs','Loss: ',epoch_loss)
            
        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
       
        feed_dict_test= {x: x_test1,
                         y: y_test1}
        
        feed_dict_train = {x: x_train1,
                     y: y_train1}
        acc = sess.run(accuracy, feed_dict = feed_dict_train)
        print("Accuracy on train-set: {0:.1%}".format(acc))
        
        acc = sess.run(accuracy, feed_dict = feed_dict_test)
        print("Accuracy on test-set: {0:.1%}".format(acc))
neural_nets_train(x_train)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

Epoch 0 out of  282 Epochs Loss:  19126.2207031
Epoch 1 out of  282 Epochs Loss:  15793.1328125
Epoch 2 out of  282 Epochs Loss:  13178.2412109
Epoch 3 out of  282 Epochs Loss:  11080.2861328
Epoch 4 out of  282 Epochs Loss:  9324.94335938
Epoch 5 out of  282 Epochs Loss:  7782.34619141
Epoch 6 out of  282 Epochs Loss:  6393.62109375
Epoch 7 out of  282 Epochs Loss:  5149.58300781
Epoch 8 out of  282 Epochs Loss:  4055.91625977
Epoch 9 out of  282 Epochs Loss:  3126.30419922
Epoch 10 out of  282 Epochs Loss:  2362.43334961
Epoch 11 out of  282 Epochs Loss:  1748.99279785
Epoch 12 out of  282 Epochs Loss:  1269.86291504
Epoch 13 out of  282 Epochs Loss:  901.412353516
Epoch 14 out of  282 Epochs Loss:  627.856140137
Epoch 15 out of  282 Epochs Loss:  430.652099609
Epoch 16 out of  282 Epochs Loss:  29

Epoch 159 out of  282 Epochs Loss:  2.47683572769
Epoch 160 out of  282 Epochs Loss:  2.47236037254
Epoch 161 out of  282 Epochs Loss:  2.46783399582
Epoch 162 out of  282 Epochs Loss:  2.46329331398
Epoch 163 out of  282 Epochs Loss:  2.458745718
Epoch 164 out of  282 Epochs Loss:  2.45563220978
Epoch 165 out of  282 Epochs Loss:  2.45280337334
Epoch 166 out of  282 Epochs Loss:  2.44996404648
Epoch 167 out of  282 Epochs Loss:  2.44711494446
Epoch 168 out of  282 Epochs Loss:  2.44426727295
Epoch 169 out of  282 Epochs Loss:  2.44143819809
Epoch 170 out of  282 Epochs Loss:  2.43858528137
Epoch 171 out of  282 Epochs Loss:  2.4356918335
Epoch 172 out of  282 Epochs Loss:  2.43279027939
Epoch 173 out of  282 Epochs Loss:  2.42990088463
Epoch 174 out of  282 Epochs Loss:  2.42700457573
Epoch 175 out of  282 Epochs Loss:  2.42410087585
Epoch 176 out of  282 Epochs Loss:  2.42118430138
Epoch 177 out of  282 Epochs Loss:  2.41825914383
Epoch 178 out of  282 Epochs Loss:  2.41533088684
Epo